# Implementing Laplace for $\epsilon$-DP

## Generating a Dummy Population

We'll create a dummy dataset with some records of fake people who we can then summarize in a some tables.

The characteristics we'll use will be: Sex, Age, Marital Status, Earnings, Occupation

In [1]:
# Some initial Python setup requirements for later code.

# Required to get the plots inline for Census implementation.
%matplotlib inline

# Load the libraries we need.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import truncnorm

# Set the threshold for numpy output values that get printed to the screen.
np.set_printoptions(threshold=10)
pd.options.display.max_rows = 10

## Population Distributions

The options we will allow for each of those categories are:
* Sex: {Male, Female}, evenly distributed.
* Age: A normal distribution of integers centered around 45 and with a StDev of 10.
* Marital Status: {Married, Widowed, Separated, Divorced, Never Marries}, based on the [ACS 1-year estimates for 2018](https://data.census.gov/cedsci/table?q=Marital%20Status%20and%20Marital%20History&hidePreview=false&t=Marital%20Status%20and%20Marital%20History&tid=ACSST1Y2018.S1201&vintage=2018)
* Earnings: A truncated log-normal distribution of integers.
* Occupation: {Statistician, Economist, Geographer, IT Specialist, Unicorn Wrangler}, with an arbitrary distribution.

In [2]:
# Specify how many dummy records we want.
numrecs=1000

# Valid Values
sexvalid=["M", "F"]
marrvalid=["M", "W", "S", "D", "N"]
occvalid=["Statistician", "Economist", "Geographer", "IT Specialist", "Unicorn Wrangler"]

# Generate arrays of random values.
sexarray = np.random.choice(a=sexvalid,size=numrecs,p=[0.50, 0.50])
agearray = np.random.normal(loc=45, scale=10, size=numrecs).round().astype(np.int)
marrarray = np.random.choice(a=marrvalid,size=numrecs,p=[0.478, 0.057,0.109,0.019,0.337])
earnarray=np.power(10,truncnorm.rvs(-3, 4, loc=5, scale=0.5, size=numrecs)).round().astype(np.int)
occarray=np.random.choice(a=occvalid,size=numrecs,p=[0.3,0.3,0.2,0.19,0.01])


# Construct the full data structure and print some records.
myrecs = pd.DataFrame({'Sex': sexarray, 
                       'Age': agearray,
                       'Married': marrarray,
                       'Earnings': earnarray,
                       'Occupation': occarray
                    }, columns=['Sex', 'Age', 'Married', 'Earnings', 'Occupation'])
print(myrecs)


    Sex  Age Married  Earnings     Occupation
0     M   45       M    213411      Economist
1     M   47       M    270500  IT Specialist
2     M   48       M   3296768     Geographer
3     F   53       M    117792   Statistician
4     M   44       M     74366   Statistician
..   ..  ...     ...       ...            ...
995   F   54       M    231524   Statistician
996   M   51       N    257908     Geographer
997   M   37       N    188947   Statistician
998   F   58       N     80671  IT Specialist
999   F   39       M    118124     Geographer

[1000 rows x 5 columns]


## Summarize Data
For this example, we want to publish a count of Occupation by Sex. For reference, we'll start by looking at the true version of the table.

In [3]:
# Note that we reindex using the full set of valid values. Unless we have structural zeros, we must also protect the real zeros.
truetab=pd.crosstab(index=myrecs['Occupation'], columns=myrecs['Sex'])
truetab=truetab.reindex(index=occvalid, columns=sexvalid, fill_value=0)
print(truetab)


Sex                 M    F
Occupation                
Statistician      137  148
Economist         148  164
Geographer         99   91
IT Specialist     106   97
Unicorn Wrangler    3    7


# Generate Laplace Noise
Now we will generate a noise value for each cell in the table too be published.

In [7]:
# First let's define the scale of the Laplace distribution in terms of sensitivity and epsilon.
sensitivity=1.
epsilon=0.5
scale=sensitivity/epsilon

print("Epsilon = {:4.2f}".format(epsilon))
print("Sensitivity = {:4.2f}".format(sensitivity) )
print("Scale = {:4.2f}".format(scale))
print()


noise = np.random.laplace(0,scale,truetab.shape)
print(noise)


Epsilon = 0.50
Sensitivity = 1.00
Scale = 2.00

[[ 2.27727214 -1.90631261]
 [-1.0745796   3.14459949]
 [ 0.9501998   2.65654731]
 [ 1.40929971 -2.98973108]
 [-0.63442318  0.12788332]]


# Create Noisy Table
Now we add true data and the noise to get a noisy table. Depending upon our publication strategy we may want to post-process the table (e.g. rounding, excluding negative values).

In [8]:
# Print the noisy table.
noisytab=truetab+noise
print(noisytab)
print()

# Post-process the noisy table to round and set negative values to zero.
noisytabpost=noisytab.round().astype(np.int)
noisytabpost[noisytabpost < 0]=0
print("Noisy table to be published:")
print(noisytabpost)
print()

print("True table for comparison:")
print(truetab)

Sex                        M           F
Occupation                              
Statistician      139.277272  146.093687
Economist         146.925420  167.144599
Geographer         99.950200   93.656547
IT Specialist     107.409300   94.010269
Unicorn Wrangler    2.365577    7.127883

Noisy table to be published:
Sex                 M    F
Occupation                
Statistician      139  146
Economist         147  167
Geographer        100   94
IT Specialist     107   94
Unicorn Wrangler    2    7

True table for comparison:
Sex                 M    F
Occupation                
Statistician      137  148
Economist         148  164
Geographer         99   91
IT Specialist     106   97
Unicorn Wrangler    3    7


# Measure the Error
Now we will calculate the $L_{1}$ error between the true table and the noisy table. First cell by cell and then in total.

In [9]:
# First cell-by-cell errors.
errortab=np.abs(noisytabpost-truetab)
print("Cell-by-cell absolute error:")
print(errortab)
print()

# Then percentage errors.
pcterrortab=errortab/truetab
print("Cell-by-cell relative error:")
print(pcterrortab)
print()

# Now the total error and the relative error.
errorsum=errortab.sum().sum()
print("Total L1 Error: {0:1d}".format(errorsum))
truesum=truetab.sum().sum()
errorrel=100*errorsum/truesum
print("Relative L1 Error: {:4.2f}%".format(errorrel))

Cell-by-cell absolute error:
Sex               M  F
Occupation            
Statistician      2  2
Economist         1  3
Geographer        1  3
IT Specialist     1  3
Unicorn Wrangler  1  0

Cell-by-cell relative error:
Sex                      M         F
Occupation                          
Statistician      0.014599  0.013514
Economist         0.006757  0.018293
Geographer        0.010101  0.032967
IT Specialist     0.009434  0.030928
Unicorn Wrangler  0.333333  0.000000

Total L1 Error: 17
Relative L1 Error: 1.70%
